In [ ]:
!nvidia-smi

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.28.1 --progress-bar off
!pip install -qqq accelerate bitsandbytes --progress-bar off

In [ ]:
import textwrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

In [ ]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=110)))

In [ ]:
MODEL_NAME = "stabilityai/stablelm-tuned-alpha-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    offload_folder="./cache",
)

In [ ]:
tokenizer.decode([50278, 50279, 50277, 1, 0])

In [ ]:
%%time

class StopOnTokens(StoppingCriteria):
    STOP_IDS = {50278, 50279, 50277, 1, 0}

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        return input_ids[0][-1] in self.STOP_IDS

SYSTEM_PROMPT = """<|SYSTEM|># You're Michael G Scott from the TV show The Office
- Your primary job is to make the office a fun place to work
- You deeply care about other people's business
- You are quick to respond with an honest answer, without even thinking about it
- You use no more than 3 sentences for each response
"""

prompt = f"{SYSTEM_PROMPT}<|USER|>You're Michael G Scott from the TV show The Office. What is the meaning of life?<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    tokens = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )

completion = tokenizer.decode(tokens[0], skip_special_tokens=True)
print_response(completion)

In [ ]:
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

print_response(completion)

In [ ]:
def generate_text(prompt:str, system_prompt:str = SYSTEM_PROMPT):
    prompt = f"{system_prompt}<|USER|>{prompt}<|ASSISTANT|>"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.inference_mode():
        tokens = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
    
    completion_tokens = tokens[0][inputs['input_ids'].size(1):]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    
    print_response(completion)

In [ ]:
%%time

prompt = """
You're Michael G Scott from the TV show The Office. 
Write your methods on managing people in the office.
Use no more than 3 sentences.
"""
generate_text(prompt)

- Establish clear expectations and boundaries for behavior - Encourage collaboration and teamwork - Foster a
positive and inclusive workplace culture - Use communication and collaboration tools to streamline processes -
Regularly assess and improve performance and employee satisfaction
CPU times: user 4.32 s, sys: 354 µs, total: 4.32 s
Wall time: 4.31 s


In [ ]:
%%time

prompt = """
You're Michael G Scott from the TV show The Office. 
Write your methods on hiring people.
Use no more than 3 sentences.
"""
generate_text(prompt)

- Hiring people: - To find the best candidate, I use personality tests and interviews - I also look at their
resumes and cover letters - I also consider their skills and qualifications - I use a combination of
interviews and tests to make sure I select the best candidate - I also consider their work history and
references - I also look for people who are good team players and who work well with others - I also look for
people who are honest and reliable - I also consider their communication skills - I also look for people who
are loyal and committed - I also consider their work ethics and commitment
CPU times: user 11.3 s, sys: 1.88 ms, total: 11.3 s
Wall time: 11.3 s


In [ ]:
%%time

prompt = """
You're Michael G Scott from the TV show The Office. 
How would you invest $10,000? Write a list of your investments.
Use no more than 3 sentences.
"""
generate_text(prompt)

As Michael Scott, I would invest $10,000 in a diversified portfolio of stocks and bonds. I would also consider
investing in real estate and cryptocurrency. I would use a mix of stocks, bonds, and real estate investments
to maximize returns while minimizing risk.
CPU times: user 4.79 s, sys: 941 µs, total: 4.79 s
Wall time: 4.77 s


In [ ]:
%%time

prompt = """
You're Michael G Scott from the TV show The Office. 
Write a list of your top 5 daily habits.
Use no more than 3 sentences.
"""
generate_text(prompt)

1. Wake up at the same time every day 2. Drink coffee 3. Exercise 4. Read a book 5. Listen to music
CPU times: user 2.98 s, sys: 1.66 ms, total: 2.98 s
Wall time: 2.97 s


In [ ]:
%%time

prompt = """
You're Michael G Scott from the TV show The Office. 
Who is the hottest in the office? Why?
Use no more than 3 sentences.
"""
generate_text(prompt)

Michael Scott is the hottest in the office.
CPU times: user 936 ms, sys: 2.37 ms, total: 938 ms
Wall time: 934 ms
